In [ ]:
import pandas
import numpy
from matplotlib import pyplot
from ipywidgets import Play, IntSlider, jslink, HBox, interactive_output

In [ ]:
filename = "pmea.csv"
data = pandas.read_csv(filename, header=0, index_col=0).T
data.index = pandas.read_csv(filename, header=None, index_col=0).iloc[0]
data.index.name = ""
data.T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])

In [ ]:
# 横軸選択
flag = False
flag = True
xmax, xmin = 1300, 1000  # 横軸範囲
buff = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
def f(i):
    y = data.iloc[i]
    pyplot.figure(figsize=(8, 6))
    pyplot.xlim(x.max(), x.min())
    pyplot.ylim(data.min().min(), data.max().max())
    pyplot.plot(x, y)
    pyplot.show()

x = data.columns
play = Play(min=0, max=len(data) - 1, step=1, interval=100)
slider = IntSlider(min=0, max=len(data) - 1, step=1)
jslink((play, "value"), (slider, "value"))
display(HBox([play, slider]), interactive_output(f, {"i": slider}))

In [ ]:
# 平滑化
from scipy.signal import savgol_filter

def f(i):
	s = savgol_filter(y, i, 2, 0)
	pyplot.xlim(x.max(), x.min())
	pyplot.ylim(y.min(), y.max())
	pyplot.scatter(x, y, s=5, c="blue")
	pyplot.plot(x, s, c="red")
	pyplot.show()
	pyplot.xlim(x.max(), x.min())
	pyplot.ylim((y.min()-y.max())/2, (y.max()-y.min())/2)
	pyplot.scatter(x, y-s, s=5, c="blue")
	pyplot.show()

x = data.columns
y = data.values[-1]
slider = IntSlider(min=3, max=51, step=2)
display(slider, interactive_output(f, {"i": slider}))

In [ ]:
from scipy.signal import savgol_filter

def f(i):
	s = savgol_filter(y, i, 2, 0)
	pyplot.xlim(x.max(), x.min())
	pyplot.ylim(y.min(), y.max())
	pyplot.scatter(x, y, s=5, c="blue")
	pyplot.plot(x, s, c="red")
	pyplot.show()
	s = savgol_filter(s, i, 2, 1)
	s = savgol_filter(s, i, 2, 0)
	s = savgol_filter(s, i, 2, 1)
	pyplot.xlim(x.max(), x.min())
	pyplot.ylim(s.min(), s.max())
	pyplot.plot(x, s, c="red")
	pyplot.show()

x = data.columns
y = data.values[-1]
slider = IntSlider(min=3, max=51, step=2)
display(slider, interactive_output(f, {"i": slider}))

In [ ]:
# 二次微分
flag = False
#flag = True
window = slider.get_interact_value()
print("window size =", x[window - 1] - x[0])
from scipy import signal
buff = data
buff = savgol_filter(buff, window, 2, 0)
buff = savgol_filter(buff, window, 2, 1)
buff = savgol_filter(buff, window, 2, 0)
buff = savgol_filter(buff, window, 2, 1)
buff = pandas.DataFrame(buff, index=data.index, columns=data.columns)
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff